In [157]:
import numpy as np
import pandas as pd
from numba import njit 
from collections import namedtuple
import random
import math
from collections import namedtuple

Point = namedtuple("Point", ['x', 'y'])

In [499]:
np.random.exponential(1000)

120.78088305266449

In [159]:
with open('tsp_5_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))

## greedy search

In [158]:
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
##a very pool greedy search
solution_points=[points[i-1]]
solution_index=[0]
available_points=list(range(1,nodeCount))

for position in range(1,nodeCount):
    better_point= available_points[0]
    dis= length(solution_points[position-1], points[better_point])
    for k in available_points:
        if length(solution_points[position-1], points[k])< dis:
            better_point= k
            dis=length(solution_points[position-1], points[k])
    solution_index.append(better_point)
    solution_points.append(points[better_point])
    available_points.pop((better_point-1))


solution_points
solution_index    
solution=solution_index           
obj = length(points[solution[-1]], points[solution[0]])
for index in range(0, nodeCount-1):
    obj += length(points[solution[index]], points[solution[index+1]])

solution

[0, 4, 3, 2, 1]

## 2-opt local search

In [162]:
##regular swap using list, instead of np.array
def swap_dist(index_1, index_2, solution):
    l= len(solution)
    curr_dist=length(points[solution[((index_1-1)%l)]], points[solution[index_1]]) + length(points[solution[((index_2+1)%l)]], points[solution[index_2]])
    swap_dist= length(points[solution[((index_1-1)%l)]], points[solution[index_2]]) + length(points[solution[((index_2+1)%l)]], points[solution[index_1]])
    return swap_dist-curr_dist


solution=list(range(nodeCount))
for i in range(10):
    for position in range(1,nodeCount):
        curr_dist=0
        best_point = solution[position]
        original_point= solution[position]
        best_index=position
        for index in range(position+1, nodeCount):
            if swap_dist(position, index, solution)<=curr_dist:
                curr_dist= swap_dist(position, index, solution)
                best_point= solution[index]
                best_index=index
        solution[position] = best_point
        solution[best_index] = original_point

# calculate the length of the tour
obj = length(points[solution[-1]], points[solution[0]])
for index in range(0, nodeCount-1):
    obj += length(points[solution[index]], points[solution[index+1]])

solution

[0, 1, 2, 3, 4]

## 2-opt search using numba

In [163]:
with open('tsp_5_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])

#     points = []
#     for i in range(1, nodeCount+1):
#         line = lines[i]
#         parts = line.split()
#         points.append(Point(float(parts[0]), float(parts[1])))

    points_np=np.zeros((nodeCount,2))
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points_np[i-1][0]= float(parts[0])
        points_np[i-1][1]= float(parts[1])
    
    solution=np.zeros(nodeCount)
    for i in range(nodeCount):
        solution[i]=i

In [267]:
import numpy as np
from numba import njit 

@njit(fastmath=True) 
def numba_swap(points_np, solution,nodeCount):
    
    ## 2-OPT swap local search:
    for position in range(1,nodeCount):
        curr_dist=0
        best_point = solution[position]
        original_point= solution[position]
        best_index=position
        for index in range(position+1, nodeCount):
            if swap_dist(position, index, solution)<=curr_dist:
                curr_dist= swap_dist(position, index, solution)
                best_point= solution[index]
                best_index=index
        solution[position] = best_point
        solution[best_index] = original_point
    
    return solution

@njit(fastmath=True) 
def swap_dist(index_1, index_2, solution):
    l= len(solution)
    curr_dist=length(points_np[int(solution[((index_1-1)%l)])], points_np[int(solution[index_1])]) + length(points_np[int(solution[((index_2+1)%l)])], points_np[int(solution[index_2])]) 
    swap_dist= length(points_np[int(solution[((index_1-1)%l)])], points_np[int(solution[index_2])])  + length(points_np[int(solution[((index_2+1)%l)])], points_np[int(solution[index_1])])
    return swap_dist-curr_dist
    
@njit(fastmath=True) 
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [173]:
for n in range(10):
    solution= numba_swap(points_np, solution,nodeCount)
    solution=solution.astype(int)
solution

array([0, 1, 2, 3, 4])

## 2-opt search using numba and simulated annealing

In [412]:
with open('tsp_5_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])
    points_np=np.zeros((nodeCount,2))
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points_np[i-1][0]= float(parts[0])
        points_np[i-1][1]= float(parts[1])
    
    solution=np.zeros(nodeCount)
    for i in range(nodeCount):
        solution[i]=i

In [493]:
@njit(fastmath=True) 
def numba_swap(points_np, solution,nodeCount, t=1):
    
    ## 2-OPT swap local search:
    for position in range(1,nodeCount):
        curr_dist=0
        best_point = solution[position]
        original_point= solution[position]
        best_index=position
        for index in range(position+1, nodeCount):
            if swap_dist(position, index, solution)<=curr_dist:
                curr_dist= swap_dist(position, index, solution)
                best_point= solution[index]
                best_index=index
            elif np.exp(-(swap_dist(position, index, solution)- curr_dist)/t)>=random.random():
                best_point= solution[index]
                best_index=index
        solution[position] = best_point
        solution[best_index] = original_point
    
    return solution

@njit(fastmath=True) 
def swap_dist(index_1, index_2, solution):
    l= len(solution)
    index_low= min(index_1, index_2)
    index_high=max(index_1, index_2)
    curr_dist=length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_low])]) + length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_high])]) 
    swap_dist= length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_high])])  + length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_low])])
    return swap_dist-curr_dist
    
@njit(fastmath=True) 
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [492]:
best_obj=100000000000000000000

for i in range(10):
    for k in range(5):
        solution= numba_swap(points_np, solution,nodeCount, (i*10+1)/500)
        solution=solution.astype(int)
        obj = length(points[solution[-1]], points[solution[0]])
        for index in range(0, nodeCount-1):
            obj += length(points[solution[index]], points[solution[index+1]])

        if obj<=best_obj:
            best_solution=[]
            for point_index in solution:
                best_solution.append(point_index)
    

# solution
best_solution

[0, 1, 2, 3, 4]

In [ ]:
best_obj=100000000000000000000
best_solution=[]
for i in range(10):
    for k in range(3):
        solution= numba_swap(points_np, solution,nodeCount, (i*8+1)/100) ### SA
        solution=solution.astype(int)
        obj = length_obj(points[solution[-1]], points[solution[0]])
        for index in range(0, nodeCount-1):
            obj += length_obj(points[solution[index]], points[solution[index+1]])

        if obj<=best_obj:
            best_solution=[]
            for point_index in solution:
                best_solution.append(point_index)

n=0
while n>2:
    solution= numba_swap(points_np, solution,nodeCount)
    solution=solution.astype(int)
    obj = length_obj(points[solution[-1]], points[solution[0]])
    for index in range(0, nodeCount-1):
        obj += length_obj(points[solution[index]], points[solution[index+1]])

    if obj<=best_obj:
        best_solution=[]
        for point_index in solution:
            best_solution.append(point_index)
    else:
        n+=1

solution= best_solution
# calculate the length of the tour
obj = length(points_np[int(solution[-1])], points_np[int(solution[0])])
for index in range(0, nodeCount-1):
    obj += length(points_np[int(solution[index])], points_np[int(solution[index+1])])
    
best_solution

In [308]:
obj = length(points[solution[-1]], points[solution[0]])
for index in range(0, nodeCount-1):
    obj += length(points[solution[index]], points[solution[index+1]])
    
obj

4.0